# 第3章: 正規表現

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み、「イギリス」に関する記事本文を表示せよ。問題21-29では、ここで抽出した記事本文に対して実行せよ。

In [1]:
import json


target_title = "イギリス"
article_text = ""

with open("jawiki-country.json", 'r') as f:
    for line in f:
        article = json.loads(line)
        if article.get("title") == target_title:
            article_text = article.get("text")
            break

# print(article_text)


## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ。

In [14]:
category_lines = []

for line in article_text.split('\n'):
    if "[[Category:" in line:
        category_lines.append(line)

print('\n'.join(category_lines))

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ。

In [15]:
import re


p = re.compile(r'Category:([^|*\]]+)')
category_names = p.findall(''.join(category_lines))
print('\n'.join(category_names))

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ。

In [4]:
import re


p = re.compile(r'^(=+)\s*(.*?)\s*\1', re.M)
level_and_section = p.findall(article_text)

for level, section_name in level_and_section[:5]:
    print(f"level: {len(level) - 1} {section_name}")

level: 1 国名
level: 1 歴史
level: 1 地理
level: 2 主要都市
level: 2 気候


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ。

In [13]:
import re


p = re.compile(r'\[\[(?:ファイル|File):\s*(.+?\.(?:jpg|jpeg|png|svg|gif|bmp|tiff|ogg))', re.IGNORECASE)
print('\n'.join(p.findall(article_text)[:3]))

Royal Coat of Arms of the United Kingdom.svg
United States Navy Band - God Save the Queen.ogg
Descriptio Prime Tabulae Europae.jpg


## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し、辞書オブジェクトとして格納せよ。

In [6]:
import re


# Firstly, obtain 「基礎情報」template.
template_p = re.compile(r'\{\{基礎情報.*\n\}\}', re.DOTALL)   # re.DOTALL -> see all the text as only one line
template_text = template_p.findall(article_text)[0]
# print(template_text)

# Secondly, extract what we need.
p = re.compile(r'\|(.*?)\s*?=\s*(.*)')
tuples = p.findall(template_text)
# print(tuples)

template_dict = {}

for key, value in tuples:
    template_dict[key] = value


i = 0
for k, v in template_dict.items():
    if i < 3:
        print(f"{k}, {v}")
    i += 1



略名, イギリス
日本語国名, グレートブリテン及び北アイルランド連合王国
公式国名, {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />


## 26. 強調マークアップの除去
25の処理時に、テンプレートの値からMediaWikiの強調マークアップ（弱い強調、強調、強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）。

In [7]:
import re


template_dict_without_emphasis = {}

for key, value in template_dict.items():
    new_key = key.strip()
    new_value = re.sub(r'\'{2,5}', '', value).strip()
    template_dict_without_emphasis[new_key] = new_value

i = 0
for k, v in template_dict_without_emphasis.items():
    if i < 3:
        print(f"{k}, {v}")
    i += 1


略名, イギリス
日本語国名, グレートブリテン及び北アイルランド連合王国
公式国名, {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />


## 27. 内部リンクの除去
26の処理に加えて、テンプレートの値からMediaWikiの内部リンクマークアップを除去し、テキストに変換せよ（参考: マークアップ早見表）。

In [ ]:
import re


template_dict_without_emphasis_and_links = {}

for key, value in template_dict_without_emphasis.items():
    new_value = re.sub(r'\[\[([^|\]]*\|)?(.*?)\]\]', r'\2', value)
    template_dict_without_emphasis_and_links[key] = new_value

i = 0
for k, v in template_dict_without_emphasis_and_links.items():
    if i < 3:
        print(f"{k}, {v}")
    i += 1

略名, イギリス
日本語国名, グレートブリテン及び北アイルランド連合王国
公式国名, {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />


## 28. MediaWikiマークアップの除去
27の処理に加えて、テンプレートの値からMediaWikiマークアップを可能な限り除去し、国の基本情報を整形せよ。

In [12]:
import re


def clean_mediawiki_markup(text):
    text = re.sub(r'(<br />)*', '', text)  # Clean <br /> markup
    text = re.sub(r'(<ref.*?>.*?</ref>|<ref>|<ref.*?\/>)', '', text)  # Clean <ref>...</ref> or <ref...> markup
    text = re.sub(r'\{\{.*?\}\}', '', text)  # Clean {{...}} markup
    return text

clean_template_dict = {}


for key, value in template_dict_without_emphasis_and_links.items():
    new_value = clean_mediawiki_markup(value)
    clean_template_dict[key] = new_value

i = 0
for k, v in clean_template_dict.items():
    if i < 3:
        print(f"{k}, {v}")
    i += 1

略名, イギリス
日本語国名, グレートブリテン及び北アイルランド連合王国
公式国名, 英語以外での正式国名:


## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し、国旗画像のURLを取得せよ。（ヒント: MediaWiki APIのimageinfoを呼び出して、ファイル参照をURLに変換すればよい）

In [56]:
import requests
from IPython.display import Image


# Obtain the file name.
filename = template_dict["国旗画像"]  # "Flag of the United Kingdom.svg"
title = f"File:{filename}"

params = {
    "action": "query",
    "titles": title,
    "prop": "imageinfo",
    "iiprop": "url",
    "format": "json"
}

url = "https://ja.wikipedia.org/w/api.php"
response = requests.get(url, params=params)
data = response.json()

pages = data['query']['pages']
for page in pages.values():
    image_url = page['imageinfo'][0]['url']
    print("国旗画像URL:", image_url)

Image(url=image_url)

国旗画像URL: https://upload.wikimedia.org/wikipedia/commons/8/83/Flag_of_the_United_Kingdom_%283-5%29.svg
